In [79]:
import sys
sys.path.append('c:\\python312\\lib\\site-packages')
import numpy as np
import numba as nb
from numba import jit
import pandas as pd
from tqdm import tqdm 
import re 
from functools import partial
from numba import njit
import time
import numpy as np
import numba
from numba import njit
from numba.typed import Dict
from numba.types import int64, unicode_type
import cupy as cp 

In [80]:
# Load the data
df = pd.read_csv("data/bbc-text.csv")

In [81]:
def create_vocabulary(complete_text):
        text = re.sub(r'[^\w\s]', ' ', complete_text).split()
        vocabulary = list(set(text))
        print("Vocabulary size: ", len(vocabulary))
        return vocabulary
complete_text = ' '.join(df["text"].tolist())
vocabulary=create_vocabulary(complete_text)
vocabulary = {word: idx for idx, word in enumerate(vocabulary)}
print(len(vocabulary))

Vocabulary size:  29457
29457


In [82]:
@njit
def numpy_matrix_test():
    A = np.zeros((12, 12))
    return A
numpy_matrix_test()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [83]:
import time

def log_time(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()  # Record start time
        result = func(*args, **kwargs)  # Execute the wrapped function
        end_time = time.time()  # Record end time
        elapsed_time = end_time - start_time
        print(f"Function '{func.__name__}' executed in {elapsed_time:.4f} seconds")
        return result
    return wrapper

In [84]:
side_window_size=3
def input_one_hot_vectors(words,vocabulary):
     
    inputs=np.zeros((len(words),len(vocabulary)),int)
     
    for i in range(len(words)): 
        inputs[i][vocabulary[words[i]]]=1
        
    return inputs 

 

@log_time 
def inputs_window_words(sequence,vocabulary,side_window_size):
    training_samples=[]
    for i in range(side_window_size,len(sequence)-side_window_size):
        words_before=sequence[i-side_window_size:i]
        words_after=sequence[i+1:i+1+side_window_size]
        #print(words_before)
        #print(words_after)
        words_input=words_before+words_after
        X_i=input_one_hot_vectors(words_input,vocabulary)
        y_i=input_one_hot_vectors([sequence[i]],vocabulary)
        #print(X_i,y_i)
        training_sample=[X_i,y_i]
        training_samples.append(training_sample)
    return training_samples



vocab_dict = {word: idx for idx, word in enumerate(vocabulary)}    
for i in range(1):
    
    training_samples=inputs_window_words(re.sub(r'[^\w\s]',' ',df["text"].iloc[i]).split(),vocab_dict,side_window_size)
    
    print(len(training_samples))

Function 'inputs_window_words' executed in 0.2666 seconds
744


In [85]:
Xi=cp.array(training_samples[0][0])
Xi.shape

(6, 29457)

In [86]:
yi=cp.array(training_samples[0][1])
yi.shape

(1, 29457)

In [103]:
embedding_size=300
semi_context_window=3

W1= cp.random.rand(len(vocabulary),embedding_size)   
B1 = cp.random.rand(embedding_size)
 
#W2 = cp.random.rand(embedding_size, len(vocabulary))
#B2 = cp.random.rand(len(vocabulary))
W2C = cp.random.rand(semi_context_window * 2 * embedding_size, len(vocabulary))
B2C = cp.random.rand(len(vocabulary))
learning_rate = 0.005

In [104]:
@log_time
def softmax(x, axis=-1):
        x = cp.clip(x, -1e4, 1e4)  # Clip for numerical stability
        e_x = cp.exp(x - np.max(x, axis=axis, keepdims=True))
        return e_x / cp.sum(e_x, axis=axis, keepdims=True)


In [105]:
sigma_Z1= softmax(cp.matmul(cp.array(Xi), W1) +  B1)
print(sigma_Z1.shape)
sigma_Z1 = sigma_Z1.reshape(1, sigma_Z1.shape[0] * embedding_size)
sigma_Z1.shape

Function 'softmax' executed in 0.0010 seconds
(6, 300)


(1, 1800)

In [106]:
sigma_Z2 = softmax(cp.matmul(sigma_Z1, W2C) + B2C)
sigma_Z2.shape

Function 'softmax' executed in 0.0000 seconds


(1, 29457)

In [109]:
@log_time
def forward(Input):
    global W1,B1,W2C,B2C
     
    
    sigma_Z1 = softmax(cp.matmul(cp.array(Input), W1) +  B1)


    if flattening_strategy == "concat":
        sigma_Z1 = sigma_Z1.reshape(1, sigma_Z1.shape[0] * embedding_size)
        sigma_Z2 = softmax( cp.matmul(sigma_Z1, W2C) + B2C)


    if flattening_strategy == "average":
        sigma_Z1 = cp.mean(sigma_Z1, axis=0).reshape(1, sigma_Z1.shape[1])
        sigma_Z2 = softmax(cp.matmul(sigma_Z1, W2) + B2)

    return [sigma_Z1, sigma_Z2]

flattening_strategy= "concat"
sigma_Z1, sigma_Z2=forward(Xi)

Function 'softmax' executed in 0.0000 seconds
Function 'softmax' executed in 0.0010 seconds
Function 'forward' executed in 0.0030 seconds


In [136]:
@log_time
def cross_entropy_loss(predictions, target):
    # Cross-entropy loss for a batch of predictions and targets
    batch_loss = -cp.sum(target * cp.log(predictions + 1e-9), axis=1)
    return cp.mean(batch_loss)

Loss=cross_entropy_loss(sigma_Z2, yi)
 
print("loss:", Loss)

dLoss_dZ2=sigma_Z2-yi

Function 'cross_entropy_loss' executed in 0.0010 seconds
loss: 10.301150539430743


In [137]:
dLoss_dZ2.shape,W2C.shape,sigma_Z1.shape,Xi.shape

((1, 29457), (1800, 29457), (1, 1800), (6, 29457))

In [143]:
#Gradient Embedding Weights C
dL_dSigmaZ1=cp.matmul(dLoss_dZ2,W2C.T)
dL_dZ1=dL_dSigmaZ1*sigma_Z1*(1-sigma_Z1)
dL_dZ1=dL_dZ1.reshape(2*semi_context_window,embedding_size)
dL_dW1=cp.matmul(dL_dZ1.T,Xi).T 
dL_dW1.shape,W1.shape

((29457, 300), (29457, 300))

In [144]:
#Gradient Embedding Bias C
dL_dB1=cp.sum(dL_dZ1,axis=0).T
dL_dB1.shape,B1.shape

((300,), (300,))

In [147]:
#Gradient Embedding W2C
dL_dW2=dLoss_dZ2.T@sigma_Z1 
dL_dW2=dL_dW2.T
dL_dW2.shape,W2C.shape

((1800, 29457), (1800, 29457))

In [156]:
#Gradient Embedding B2C
dL_dB2=dLoss_dZ2.T 
dL_dB2.shape,B2.reshape(B2.shape[0]).shape

((29457, 1), (29457,))

In [152]:
#Gradient Embedding Weights
dL_dSigmaZ1=dLoss_dZ2@W2.T
dL_dSigmaZ1=dL_dSigmaZ1.repeat(6,axis=0)
dL_dZ1=dL_dSigmaZ1*sigma_Z1*(1-sigma_Z1)
dL_dW1=dL_dZ1.T@Xi
dL_dW1=dL_dW1.T

ValueError: operands could not be broadcast together with shapes (6, 300) (1, 1800)

In [75]:
#Gradient Embedding Bias
dL_dSigmaZ1=dLoss_dZ2@W2.T
dL_dSigmaZ1=dL_dSigmaZ1.repeat(6,axis=0)
dL_dZ1=dL_dSigmaZ1*sigma_Z1*(1-sigma_Z1)
dL_dB1=cp.sum(dL_dZ1,axis=0).T
dL_dB1.shape,B1.shape

((300,), (300,))

In [77]:
#Gradient Embedding W2
dL_dW2=dLoss_dZ2.T@sigma_Z1 
dL_dW2=dL_dW2.T
dL_dW2.shape,W2.shape

((300, 29457), (300, 29457))

In [149]:
#Gradient Embedding B2
dL_dB2=dLoss_dZ2 
dL_dB2=dL_dB2.T
dL_dB2.shape,B2.shape

((29457, 1), (29457,))

In [150]:
dL_dB2[0]

array([2.1311914e-05])

In [151]:
import cupy as cp


@log_time
def fast_backpropagation_gpu(dLoss_dZ2, sigma_Z_1, sigma_Z_2, Xi,
                             words_len_embedding_layer,
                             words_len_embedding_bias,
                             outlayer_maps_vocab_concat,
                             out_bias_maps_vocab_concat,
                             embedding_size,
                             semi_context_window,
                             learning_rate):
    
    # Convert everything to CuPy arrays to leverage GPU acceleration
    

    if flattening_strategy == "concat":
        # Matrix multiplication on GPU
        dLoss_dW2 = cp.matmul(sigma_Z_1.T, dLoss_dZ2)  # (1800, 29457)

        # Gradient of loss with respect to b2
        dLoss_db2 = cp.sum(dLoss_dZ2, axis=0)  # (29457,)

        # Gradient of loss with respect to Z1
        dLoss_dZ1 = cp.matmul(dLoss_dZ2, outlayer_maps_vocab_concat.T)  # (1, 1800)

        # Softmax gradient with element-wise operations on GPU
        dZ1_dPreSoftmax = sigma_Z_1 * (1 - sigma_Z_1)
        dLoss_dPreSoftmax = dLoss_dZ1 * dZ1_dPreSoftmax

        # Gradient of loss with respect to W1
        dLoss_dW1 = cp.matmul(Xi.T, dLoss_dPreSoftmax.reshape(Xi.shape[0], -1))  # (29457, 1800)

        # Gradient of loss with respect to b1
        dLoss_db1 = cp.sum(dLoss_dPreSoftmax, axis=(0, 1))  # (300,)

        # Update weights and biases (in-place operations on GPU)
        outlayer_maps_vocab_concat -= learning_rate * dLoss_dW2
        out_bias_maps_vocab_concat -= learning_rate * dLoss_db2
        words_len_embedding_layer -= learning_rate * dLoss_dW1
        words_len_embedding_bias -= learning_rate * dLoss_db1

   



learning_rate=0.01   
a=fast_backpropagation_gpu(dLoss_dZ2,sigma_zout_one,sigma_zout_output,Xi,
               words_len_embedding_layer,
                words_len_embedding_bias,
                outlayer_maps_vocab_concat,
                out_bias_maps_vocab_concat,
                embedding_size,
                semi_context_window,
                learning_rate)    

NameError: name 'sigma_zout_one' is not defined

In [ ]:
print(cp.cuda.runtime.getVersion())

In [8]:
for i in range(len(df)):
    training_samples=inputs_window_words(re.sub(r'[^\w\s]',' ',df["text"].iloc[i]).split(),vocabulary,side_window_size)
    print(len(training_samples))

NameError: name 'inputs_window_words' is not defined

In [ ]:
for i in range(len(df)):
    start = time.time()
    training_samples=inputs_window_words(re.sub(r'[^\w\s]',' ',df["text"].iloc[i]).split(),vocabulary,side_window_size)
    end = time.time()
    print(len(training_samples)," delta t",end - start)

In [4]:
import time

start = time.time()
print("hello")
end = time.time()
print(end - start)

def input_one_hot_vectors(words,vocabulary):
    #print(len(words))
    inputs=np.zeros((len(words),len(vocabulary)),int)
    #print(inputs.shape)
    for i in range(len(words)): 
        inputs[i][vocabulary.index(words[i])]=1
    return inputs
side_window_size=3

 
def inputs_window_words(sequence,vocabulary,side_window_size):
   
    training_samples=[]
    for i in range(side_window_size,len(sequence)-side_window_size):
        words_before=sequence[i-side_window_size:i]
        words_after=sequence[i+1:i+1+side_window_size]
        #print(words_before)
        #print(words_after)
        words_input=words_before+words_after
        X_i=input_one_hot_vectors(words_input,vocabulary)
        y_i=input_one_hot_vectors([sequence[i]],vocabulary)
        #print(X_i,y_i)
        training_sample=[X_i,y_i]
        training_samples.append(training_sample)
    return training_samples
    
 
for i in range(len(df)):
    start = time.time()
    training_samples=inputs_window_words(re.sub(r'[^\w\s]',' ',df["text"].iloc[i]).split(),vocabulary,side_window_size)
    end = time.time()
    print(len(training_samples)," delta t",end - start)

hello
0.0
744  delta t 3.73740816116333
294  delta t 1.3751134872436523
242  delta t 1.2456028461456299


KeyboardInterrupt: 

In [26]:
from numba import njit
import numpy as np
@njit
def foo(x):
   lst = []
   for i in range(x):
       lst.append(np.zeros(5))
   return lst

foo(4)
 

[array([0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0.])]

In [24]:
np.zeros((4,3),int)

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])